In [ ]:
from sbcbinaryformat import Streamer, Writer
import numpy as np

from GetEvent import GetEvent
import matplotlib as mpl
import matplotlib.pyplot as plt

from PIL import Image
import os

In [ ]:
plt.rc('font', size=14)

In [ ]:
RUNDIR = "/exp/e961/data/SBC-25-daqdata/"

In [ ]:
runs = [f for f in os.listdir(RUNDIR) if f.startswith("20250718") and "0" in os.listdir(RUNDIR + f)]

In [ ]:
frames = [
    [],
    [],
    []
]

for r in runs:
    data = GetEvent(RUNDIR + r, 0, "cam", strictMode=False)
    for c in range(3):
        camera = data["cam"]["c%i" % (c+1)]
        thisframes = []
        for key in camera.keys():
            if key.startswith("frame"):
                frameind = int(key[-1])
                thisframes.append((frameind, camera[key], key))

        frames[c].append([None for _ in thisframes])
        for i, v, key in thisframes:
            frames[c][-1][i] = (v, r, key)

In [ ]:
RMSs = [
    [],
    [],
    [],
]

frame_evts = [
    [],
    [],
    [],
]

for iframe, eventlist in enumerate(frames):
    for event in eventlist:
        for lo, hi in zip(event[:-1], event[1:]):
            lo_v = lo[0]
            hi_v = hi[0]
            
            RMS = np.sqrt(((hi_v - lo_v)**2).mean())
            RMSs[iframe].append(RMS)
            frame_evts[iframe].append((lo[1], lo[2], hi[2]))

RMSs = [np.array(R) for R in RMSs]

In [ ]:
_ = plt.hist(RMSs, histtype="step", linewidth=2, label=["Camera 1", "Camera 2"])
plt.ylabel("Number of Frame Differences")
plt.xlabel("Average RMS Pixel Difference")
plt.legend()

In [ ]:
# BADIND = np.argmax(RMSs[0])

BADIND = np.argsort(RMSs[0])[-2]

print(frame_evts[0][BADIND])
bad = frame_evts[0][BADIND]
badrun = bad[0]

data = GetEvent(RUNDIR + badrun, 0, "cam", strictMode=False)
plt.figure(0)
plt.imshow(data["cam"]["c1"][bad[1]])
plt.figure(1)
plt.imshow(data["cam"]["c1"][bad[2]])
# plt.figure(2)
# plt.imshow(data["cam"]["c1"][bad[1]])